In [3]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

def extract_id(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    result = []

    for item in data:
        if 'id' in item:
            result.append(item['id'])

    return result

def download_info(driver, id):
    retry_count = 1
    while retry_count > 0:
        try:
            driver.get(f'https://m.place.naver.com/place/{id}/home')
            driver.implicitly_wait(2)
            break
        except Exception:
            retry_count -= 1
            driver.refresh()

    elements_to_click = [
        '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.pSavy > div > a > div > div > span',
        '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div:nth-child(8) > div > a > span.rvCSr'
    ]
    
    elements_to_append = [
        '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div.O8qbU.pSavy > div > a *',
        '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div.O8qbU.WTjuc > div > ul a',
        '#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(8) > div > a > span.zPfVt'
    ]

    for element_selector in elements_to_click:
        element = driver.find_element(By.CSS_SELECTOR, element_selector)
        if element:
            element.click()
            for element_selector in elements_to_append:
                hospital_info_elements = []
                elements = driver.find_elements(By.CSS_SELECTOR, element_selector)
                if elements:
                    hospital_info_elements.append([element.text for element in elements])
        else:
            pass

    return hospital_info_elements

json_file_path = 'naver_total.json'
extracted_id = extract_id(json_file_path)

with webdriver.Chrome() as driver:
    results = {}

    for id in tqdm(extracted_id, desc='Downloading info'):
        result = download_info(driver, id)
        if result is not None:
            results[id] = result
            
json_data = json.dumps(results, indent=4)

with open('hospital_info.json', 'w') as file:
    file.write(json_data)

for id, info in list(results.items())[:10]:
    print(id, info)